# Checking the accuracy and conistancy of stats collected

In order to determine and test the accuracy of my ScrapeStats algorithm, I read through each teams collected seasons, tally the total amount of offensive points scored, then compare my calculated value with statmuse's database stored at https://www.statmuse.com/nfl/ask/most-total-nfl-points-last-8-years. Thus I am directly testing whether I accurately pulled each games data, as flipping the home and away team will introduce unnecessary error. 

### Reading in functions and libraries

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np


#read in returnFullSeasonStats function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import returnFullSeasonStats # type: ignore

#read in returnSpecificRow function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import returnSpecificRow # type: ignore

#read in extractNumberFromString function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import extractNumberFromString # type: ignore

#read in getSpreadandScores function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import getSpreadandScores # type: ignore

In [12]:
#directory where Stats was stored
directory = 'Stats/'

yearArr = [2017,2018,2019,2020,2021,2022,2023, 2024]

#array of each directory within stats
TeamNamesArr= os.listdir(directory)

totalPointsScoredArr = []

for team in TeamNamesArr:
    if team == ".DS_Store":
        continue
    currFullSeasonStats, currFullSeasonInfo = returnFullSeasonStats(team, yearArr)
    teamBeatSpread, scoreDifference, vegasSpread, teamScoreFloat, oppScoreFloat = getSpreadandScores(currFullSeasonStats, currFullSeasonInfo, team)
    totalPointsScoredArr.append(tuple(team,np.sum(teamScoreFloat)))


NameError: name 'pd' is not defined